# Model 000 Basic: Coupled Steady-State Solution

This model shows example usage of the Basic model from the TerrainBento package.

This model has stream power and linear diffusion of the forms:
$$\frac{d\eta}{dt}=-KA^mS^n$$
and
$$\frac{d\eta}{dt} = -D\nabla^2\eta$$

Refer to [REF MANUSCRIPT] for the definition of all symbols. 

This notebook (a) shows the initialization and running of this model, (b) saves a NetCDF file of the topography, which we will use to make an oblique Paraview image of the landscape, and (c) creates a slope-area plot at steady state.

In [1]:
# import required modules
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
%matplotlib inline

from landlab import imshow_grid

from terrainbento import Basic

In [2]:
# create the parameter dictionary needed to instantiate the model 
params = {'number_of_node_rows' : 100,
          'number_of_node_columns' : 160,
          'node_spacing' : 10.0,
          'east_boundary_closed' : False,
          'north_boundary_closed' : False,
          'west_boundary_closed' : False,
          'south_boundary_closed' : False,
          'dt' : 10.0,
          'K_sp' : 0.001,
          'm_sp' : 0.5,
          'n_sp' : 1.0,
          'linear_diffusivity' : 0.2,
          'outlet_lowering_rate' : 0.0005,
          'output_filename': 'model_000_output'
}

In [ ]:
# initialize the model by passing the parameter dictionary
basic = Basic(params=params)

# set a tolerance for determining steady state
tolerance = 0.0001

In [ ]:
# run the model

# set dt and initialize the elapsed_time variable
dt = 10 # units are in years
elapsed_time = 0 

# initialize a variable to identify if steady state has been reached
keep_running = True

# run the model until stead state has been reached. 
while keep_running == True:
    
    # save the pre-model-step topography to assess steady state. 
    pre_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    
    # run the model one step
    basic.run_one_step(dt)
    
    # save the topography post model step to assess steady state. 
    post_topo = basic.grid.at_node['topographic__elevation'][basic.grid.core_nodes]
    
    # increment elapsed time
    elapsed_time += dt
    
    # assess if steady state has been reached. 
    if max(abs(pre_topo - post_topo)) <= tolerance: # 1 mm
        keep_running = False
    
    # if time rounds to 1000 years, print out a log message. 
    if elapsed_time % 1000 == 0:
        print('Elapsed time: ', elapsed_time)
        print('Maximum Topographic Change/Tolerance: ', np.round(max(abs(pre_topo - post_topo))/tolerance, decimals=2))       

Elapsed time:  1000
Maximum Topographic Change/Tolerance:  460.05
Elapsed time:  2000
Maximum Topographic Change/Tolerance:  290.87
Elapsed time:  3000
Maximum Topographic Change/Tolerance:  102.61
Elapsed time:  4000
Maximum Topographic Change/Tolerance:  93.92
Elapsed time:  5000
Maximum Topographic Change/Tolerance:  322.27
Elapsed time:  6000
Maximum Topographic Change/Tolerance:  111.97
Elapsed time:  7000
Maximum Topographic Change/Tolerance:  151.92
Elapsed time:  8000
Maximum Topographic Change/Tolerance:  466.0
Elapsed time:  9000
Maximum Topographic Change/Tolerance:  72.99
Elapsed time:  10000
Maximum Topographic Change/Tolerance:  70.25
Elapsed time:  11000
Maximum Topographic Change/Tolerance:  42.65
Elapsed time:  12000
Maximum Topographic Change/Tolerance:  59.78
Elapsed time:  13000
Maximum Topographic Change/Tolerance:  65.53
Elapsed time:  14000
Maximum Topographic Change/Tolerance:  68.85
Elapsed time:  15000
Maximum Topographic Change/Tolerance:  83.14
Elapsed time:

In [ ]:
# MAKE SLOPE-AREA PLOT

# assign area_array and slope_array
area_array = basic.grid.at_node['drainage_area'][(basic.grid.node_x > 10)&(basic.grid.node_x < 1580)&(basic.grid.node_y >10)&(basic.grid.node_y<980)]
slope_array = basic.grid.at_node['topographic__steepest_slope'][(basic.grid.node_x > 10)&(basic.grid.node_x < 1580)&(basic.grid.node_y >10)&(basic.grid.node_y<980)]

# instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

# create an array for the detachment-limited analytical solution
u = params['outlet_lowering_rate']# m/yr, uplift or baselevel lowering rate
k = params['K_sp'] # fluvial erodibility
m = params['m_sp'] # discharge exponent
n = params['n_sp'] # slope exponent

# calculate analytical slope from area field
analytical_slope_array = np.power((u / k), 1 / n) * np.power(area_array, -m/n)

# plot the analytical solution
slope_area.plot(area_array, analytical_slope_array, linestyle='-',
                color='grey', linewidth = 1, label = 'Analytical solution')

# plot the data
slope_area.scatter(area_array, slope_array, marker='o', c='k', 
                   label = 'Numerical solution') #plot HA data
                   
# make axes log and set limits
slope_area.set_xscale('log')
slope_area.set_yscale('log')

slope_area.set_xlim(9*10**1, 3*10**5)
slope_area.set_ylim(1e-3, 1e-1)

# set x and y labels
slope_area.set_xlabel(r'Drainage area [m$^2$]')
slope_area.set_ylabel('Channel slope [-]')

# slope_area.legend(scatterpoints=1,prop={'size':12})
slope_area.tick_params(axis='x', which='major', pad=7)

# save out an output figure
output_figure = os.path.join('output_netcdfs/maintext_basic_streampower_slope_area_rev1.eps')
fig.savefig(output_figure, bbox_inches='tight', dpi=1000) #save figure

In [ ]:
#export a NetCDF of the final topography for Paraview to use.
from landlab.io.netcdf import write_netcdf
output_file = os.path.join('output_netcdfs/maintext_basic_stream_power_rev1.nc')
write_netcdf(output_file, basic.grid, format='NETCDF3_64BIT', names='topographic__elevation')

In [ ]:
# make a plot of the final steady state topography
imshow_grid(basic.grid, 'topographic__elevation')